In [ ]:
#default_exp core

# core


> Monkey patches for pandas.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
from functools import partial
from itertools import chain
import math

from fastcore.all import *
import pandas as pd
import ipywidgets as widgets
from IPython.display import display

## Utils

In [ ]:
#export
def dummydf(): return pd.DataFrame({'col_1': range(100, 105), 'col_2': ['a','b','c','d','e']})

## Transformations

In [ ]:
#export
@patch
def repetitions(self:pd.DataFrame, col): return self.groupby(col).size()

In [ ]:
#export
add_docs(pd.DataFrame,
         repetitions='Counts the number of repetitions for each element.',
         ffill=pd.core.generic.NDFrame.ffill.__doc__,
         bfill=pd.core.generic.NDFrame.bfill.__doc__,
         clip=pd.core.generic.NDFrame.clip.__doc__,
         interpolate=pd.core.generic.NDFrame.interpolate.__doc__,
         where=pd.core.generic.NDFrame.where.__doc__,
         mask=pd.core.generic.NDFrame.mask.__doc__)

In [ ]:
nodoc = [c for n,c in vars(pd.DataFrame).items() if callable(c) and not n.startswith('_') and c.__doc__ is None]

In [ ]:
show_doc(pd.DataFrame.repetitions)

<h4 id="DataFrame.repetitions" class="doc_header"><code>DataFrame.repetitions</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>DataFrame.repetitions</code>(**`col`**)

Counts the number of repetitions for each element.

In [ ]:
df = pd.DataFrame({'a': [1, 2, 3, 4, 4, 5, 5, 6, 6, 6], 'b':[1, 1, 1, 1, 2, 2, 2, 3, 3, 4]})
df.repetitions('b')

b
1    4
2    3
3    2
4    1
dtype: int64

In [ ]:
test(df.repetitions('b'), pd.Series({1:4, 2:3,3:2, 4:1}), all_equal)

In [ ]:
#export
@patch
def repetition_counts(self:pd.DataFrame, col): return self.repetitions(col).value_counts()

In [ ]:
#export
add_docs(pd.DataFrame,repetition_counts='Counts the number of groups with the same number of repetitions.')

In [ ]:
show_doc(pd.DataFrame.repetition_counts) 

<h4 id="DataFrame.repetition_counts" class="doc_header"><code>DataFrame.repetition_counts</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>DataFrame.repetition_counts</code>(**`col`**)

Counts the number of groups with the same number of repetitions.

In the following example there are three groups with one element, two groups with two elements, and one group with three elements.

In [ ]:
df.repetition_counts('a')

1    3
2    2
3    1
dtype: int64

In [ ]:
test(df.repetition_counts('a'), pd.Series({1: 3, 2:2, 3:1}), all_equal)

In [ ]:
#export
@patch
def single_events(self:pd.DataFrame, col): return self.set_index(col).loc[self.repetitions(col)==1].reset_index()

In [ ]:
#export
add_docs(pd.DataFrame, single_events='Returns rows that appear only once.')

In [ ]:
show_doc(pd.DataFrame.single_events)

<h4 id="DataFrame.single_events" class="doc_header"><code>DataFrame.single_events</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>DataFrame.single_events</code>(**`col`**)

Returns rows that appear only once.

In [ ]:
df.single_events('a')

,a,b
0,1,1
1,2,1
2,3,1


In [ ]:
test_eq(df.single_events('a'), df.loc[[0, 1, 2]])

## Functions as methods


Pandas functions that are easier to execute as DataFrame/Series methods.


In [ ]:
#export
@patch
@delegates(pd.crosstab)
def crosstab(self:pd.DataFrame, index, column, **kwargs): return pd.crosstab(self[index], self[column], **kwargs)

In [ ]:
#export
add_docs(pd.DataFrame, crosstab=pd.crosstab.__doc__)

In [ ]:
#export
@patch
def len(self:pd.DataFrame): return len(self)

In [ ]:
#export
add_docs(pd.DataFrame, len=len.__doc__)

In [ ]:
#export
@patch
def len(self:pd.Series): return len(self)

## One-liners

> These methods allow fast exploration of the data in one line.

In [ ]:
#export
@patch(as_prop=True)
def l(self:pd.Index): return L(self, use_list=True)

In [ ]:
#export
@patch(as_prop=True)
def minmax(self:pd.Series): return (self.min(), self.max())

In [ ]:
#export
@patch
def page(self:pd.DataFrame, page, page_size=5): return self.head(page*page_size).tail(page_size)

In [ ]:
#export
add_docs(pd.DataFrame, page='Shows rows from page*page_size to (page+1)*page_size')

In [ ]:
df = pd.DataFrame({'a': range(15), 'b': range(15)})
df.page(2)

,a,b
5,5,5
6,6,6
7,7,7
8,8,8
9,9,9


In [ ]:
#export
@patch
def page(self:pd.Series, page, page_size=5): return self.head(page*page_size).tail(page_size)

In [ ]:
#export
add_docs(pd.Series, 
         page='Shows rows from page*page_size to (page+1)*page_size',
         ffill=pd.core.generic.NDFrame.ffill.__doc__,
         bfill=pd.core.generic.NDFrame.bfill.__doc__,
         clip=pd.core.generic.NDFrame.clip.__doc__,
         interpolate=pd.core.generic.NDFrame.interpolate.__doc__,
         where=pd.core.generic.NDFrame.where.__doc__,
         mask=pd.core.generic.NDFrame.mask.__doc__,
         len=len.__doc__)

In [ ]:
s = pd.Series(range(15))
s.page(2)

5    5
6    6
7    7
8    8
9    9
dtype: int64

## Method Variations


These methods are slight variations from DataFrame ones.


In [ ]:
#export
@patch
def renamec(self:pd.DataFrame, d, *args):
    if args:
        if isinstance(d, dict): d = chain(*d.items())
        d = dict(chunked(listify(d) + listify(args), 2))
    return self.rename(columns=d)

In [ ]:
df = dummydf()
df.renamec({'col_1': 'col_a'}, 'col_2', 'bar')

,col_a,bar
0,100,a
1,101,b
2,102,c
3,103,d
4,104,e


In [ ]:
#export
add_docs(pd.DataFrame, renamec='Renames column names.')

In [ ]:
#export
@patch
def notin(self:pd.Series, values): return ~self.isin(values)

In [ ]:
#export
add_docs(pd.Series, notin='Whether elements in Series are not contained in `values`.')

In [ ]:
#export
@patch
def mapk(self:pd.Series, fun, **kwargs): return self.map(partial(fun, **kwargs))

In [ ]:
#export
add_docs(pd.Series, mapk='Like map but passes kwargs to function.')

## Display

In [ ]:
#export
@patch
def title(self:pd.DataFrame, title):
    '''Displays DataFrame with a title.'''
    out = widgets.Output()
    with out: display(self)
    layout = widgets.Layout(align_items='center')
    return widgets.VBox([widgets.Label(title, layout=layout), out])

In [ ]:
df = pd.DataFrame({'a': range(5), 'b': range(5)})
df.title('I am a table')

In [ ]:
#export
class Walker:
    def __init__(self, val=0, min_val=None, max_val=None): store_attr()
    def _next(self, *args, val=1, **kwargs):
        self.val += val
        if self.max_val: self.val = min(self.max_val, self.val)
    def _prev(self, *args, val=1, **kwargs):
        self.val -= val
        if self.min_val: self.val = max(self.min_val, self.val)
    def _head(self):
        self.val = 0
        if self.min_val: self.val = max(self.min_val, self.val)       

### Less

In [ ]:
#export
class Less:
    def __init__(self, df, page_size=5, page=1, where=None):
        store_attr()
        if not where is None:
            page = math.floor(L(range_of(df))[where][0]/page_size+1)
            
        self.out = widgets.Output(wait=True)
        self.out_df = widgets.Output(wait=True)
        self.out_df.append_display_data(self.df.page(page, page_size=self.page_size))
        
        self.n_pages = len(df)//self.page_size+1
        self.page = Walker(val=page, min_val=1, max_val=self.n_pages)

        self.next = widgets.Button(description='next')
        self.next.on_click(self.page._next)
        self.next.on_click(self.refresh)

        self.prev = widgets.Button(description='prev')
        self.prev.on_click(self.page._prev)
        self.prev.on_click(self.refresh)
        
        layout = widgets.Layout(width='100%', display='flex', align_items='center')
        self.out_label = widgets.Output(wait=True)
        with self.out_label: display(widgets.Label(f"page {self.page.val} of {self.n_pages}"))
        self.box = widgets.VBox([widgets.HBox([self.prev, self.next, self.out_label]), self.out_df])
        with self.out: 
            display(self.box)

    def refresh(self, event):
        self.out_df.clear_output()
        with self.out_df: display(self.df.page(self.page.val, page_size=self.page_size))
        self.out_label.clear_output()
        with self.out_label: display(widgets.Label(f"page {self.page.val} of {self.n_pages}"))

In [ ]:
#export
@patch
def less(self:pd.DataFrame, page_size=5, page=1, where=None):
    return Less(self, page_size=page_size, page=page, where=where).out

In [ ]:
#export
@patch
def to_percent(self:pd.DataFrame, exclude=[]):
    cols = self.dtypes[self.dtypes=='float'].index
    cols = [c for c in cols if c not in exclude]
    return self.style.format({c: '{:.1%}'.format for c in cols})

In [ ]:
#export
add_docs(pd.DataFrame,
         less='Displays one page of the DataFrame and buttons to move forward and backward.',
         to_percent='Formats float columns to percentage.')

In [ ]:
df = pd.DataFrame({'a': range(17), 'b': range(17)})
df.less(page_size=7, page=2)

Output()

In [ ]:
df.less(page_size=3, where=df.a.gt(5))

Output()

In [ ]:
#export
@patch
def less(self:pd.Series, page_size=5): return Less(self, page_size=page_size).out

In [ ]:
#export
add_docs(pd.Series, 
         less='Displays one page of the Series and buttons to move forward and backward.',
         )

In [ ]:
s = pd.Series(range(7))
s.less()

Output()

## Move columns to the front/back

In [ ]:
#export
@patch
def c2back(self:pd.DataFrame, cols2back):
    if not is_listy(cols2back): cols2back = [cols2back]
    cols = [c for c in self.columns if c not in cols2back]+cols2back
    return self[cols]

In [ ]:
#export
@patch
def c2front(self:pd.DataFrame, cols2front):
    if not is_listy(cols2front): cols2front = [cols2front]
    cols = cols2front + [c for c in self.columns if c not in cols2front]
    return self[cols]

In [ ]:
#export
add_docs(pd.DataFrame,
         c2back="Move columns to back",
         c2front="Move columns to front")

In [ ]:
df = dummydf()

In [ ]:
df.c2back(['col_1'])

,col_2,col_1
0,a,100
1,b,101
2,c,102
3,d,103
4,e,104


In [ ]:
df.c2back('col_1')

,col_2,col_1
0,a,100
1,b,101
2,c,102
3,d,103
4,e,104


In [ ]:
df.c2front('col_2')

,col_2,col_1
0,a,100
1,b,101
2,c,102
3,d,103
4,e,104


In [ ]:
df.c2front(['col_2'])

,col_2,col_1
0,a,100
1,b,101
2,c,102
3,d,103
4,e,104


In [ ]:
#export
@patch
def reorderc(sefl:pd.DataFrame, to_front=[], to_back=[]):
    '''Reorder DataFrame columns.'''
    return df.c2front(to_front).c2back(to_back)

In [ ]:
df['col_3'] = df['col_1']
df.reorderc(['col_3'], ['col_1'])

,col_3,col_2,col_1
0,100,a,100
1,101,b,101
2,102,c,102
3,103,d,103
4,104,e,104


## Export

In [ ]:
from nbdev.export import notebook2script; notebook2script()

Converted 00_core.ipynb.
Converted index.ipynb.
